In [ ]:
%pylab inline

In [ ]:
import scape
import katdal
import katpoint
import katsdpscripts
import glob
import re
import pandas as pd 

import csv
import operator

In [ ]:
from os import system

In [ ]:
print "Scape version: %s"%scape.__version__
print "katdal version: %s"%katdal.__version__
print "katpoint version: %s"%katpoint.__version__
print "katsdpscripts version: %s"%katsdpscripts.__version__

In [ ]:
h5_filename ='/var/kat/archive3/data/MeerKATAR1/telescope_products/2018/04/13/1523636247.h5'

%set_env h5_filename = $h5_filename

In [ ]:
res_h='




'
print res_h


In [ ]:
res_h.splitlines()

In [ ]:
delays_h={}
for line in res_h.splitlines():
    if line.startswith('Antenna'):
        ant=line[8:12]
        delays_h[ant] = {}
    if line.startswith('E (m)'):
        delays_h[ant]['E']=float(line.split(':')[1].lstrip().split(' ')[0])
    if line.startswith('N (m)'):
        delays_h[ant]['N']=float(line.split(':')[1].lstrip().split(' ')[0])
    if line.startswith('U (m)'):
        delays_h[ant]['U']=float(line.split(':')[1].lstrip().split(' ')[0])
    if line.startswith('Cable'):
        delays_h[ant]['D']=float(line.split(':')[1].lstrip().split(' ')[0])
#print delays_h

In [ ]:
res_v='

'
print res_v

In [ ]:
delays_v={}
for line in res_v.splitlines():
    if line.startswith('Antenna'):
        ant=line[8:12]
        delays_v[ant] = {}
    if line.startswith('E (m)'):
        delays_v[ant]['E']=float(line.split(':')[1].lstrip().split(' ')[0])
    if line.startswith('N (m)'):
        delays_v[ant]['N']=float(line.split(':')[1].lstrip().split(' ')[0])
    if line.startswith('U (m)'):
        delays_v[ant]['U']=float(line.split(':')[1].lstrip().split(' ')[0])
    if line.startswith('Cable'):
        delays_v[ant]['D']=float(line.split(':')[1].lstrip().split(' ')[0])
print delays_v

# Cross check between polarizations for derived positions

In [ ]:
for a in sorted(delays_v):
    print a,
    print("diff E %06.3f") %(delays_h[a]['E'] - delays_v[a]['E']),
    print("diff N %06.3f") %(delays_h[a]['N'] - delays_v[a]['N']),
    print("diff U %06.3f") %(delays_h[a]['U'] - delays_v[a]['U']),
    print("diff D %06.3f") %(delays_h[a]['D'] - delays_v[a]['D'])
    
# expect differences between E,N and U less than 1cm and Delays less than 3m 

In [ ]:
# write solutions to file
matchRe = re.findall('[0-9]{10,}', h5_filename)
filename='delay_file_'+''.join(matchRe)+'.txt'

with open(filename, 'w') as f:
    for h,v in zip(delays_h,delays_v):
        print >> f, h,v, mean([delays_h[h]['E'],delays_v[v]['E']]) , \
                    mean([delays_h[h]['N'],delays_v[v]['N']]), \
                    mean([delays_h[h]['U'],delays_v[v]['U']]), \
                    delays_h[h]['D'],delays_v[v]['D']
    f.close()

In [ ]:
# sort delay file by antenna name and load antennaLOC
sample = open(filename, 'r')
sorter = csv.reader(sample, delimiter=' ')
sort = sorted(sorter, key=operator.itemgetter(1))

with open('ant_loc.csv', 'r') as f:
    tocs = f.read()

In [ ]:
cols = ['Ants', 'Delay', 'East', 'North', 'UP', 'TOC']
df = pd.DataFrame(columns=cols)
for line in sort:
    p = re.compile(line[0]+',\s+\d{4}\.\d{1,3}')
    toc = float(p.search(tocs).group()[6:])
    df = df.append(pd.Series(
                             [
                                line[0],
                                line[-2],
                                line[2],
                                line[3],
                                line[4], 
                                toc
                             ], 
                              index=['Ants', 'Delay', 'East', 'North', 'UP', 'TOC']
                            ), ignore_index = True)

In [ ]:
deltaTOC, deltaUP = [], []
deltaTOC.append(0)
deltaUP.append(0)
for i in range(1, len(df.TOC[1:])+1):
    deltaTOC.append(df.TOC[i] - df.TOC[i-1])
    deltaUP.append(float(df.UP[i]) - float(df.UP[i-1]))

em  = [' ' for i in range(len(deltaUP))]
df[' '] =  pd.Series(em,  index = df.index)

df['delta UP']  = pd.Series(deltaUP,  index = df.index)
df['delta TOC'] = pd.Series(deltaTOC, index = df.index)

print('\nDelay Solutions and antenna locations\n')
df

In [ ]:
# send file to ops server to process and push to github

opsServer='ops.kat.ac.za'
rPath='/home/kat/ops_team_sw/utilities/update_delay_models'
cmd = "sshpass -p {} scp -r {} kat@{}:{}".format('kat', filename, opsServer, rPath)
system(cmd)
